## Альтернативный подход к задачам мультиклассовой классификации:

Вместо того, чтобы предсказывать таргет напрямую, мы можем закодировать таргет в двоичном коде и предсказывать каждый бит по отдельности.

Тогда нам понадобится всего $~ \text{log}_2(L + 1)$ классификаторов на L + 1 класс.

Попробуем обучить набор таких логрегов и сравнить качество полученного классификатора с мультиномиальной и OvR регрессиями.

In [1]:
# импорт базовых библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set();
import scipy.stats as sps

#### Шаг 0: импорт и препроцессинг данных

In [2]:
%%time
# выгружаем датасет
from sklearn.datasets import fetch_openml
mnist = fetch_openml(data_id=554) # https://www.openml.org/d/554
# генерируем сегментирующую случайную переменую
rn = pd.Series(sps.randint.rvs(1, 101, size = len(mnist.data), random_state = 42))
# разбиваем на трейн/валидацию/тест
X = mnist.data
y = mnist.target
train_mask, val_mask, test_mask = (rn <= 60), ((rn > 60) & (rn <= 70)), (rn > 70)
X_train, y_train, X_test, y_test, X_val, y_val = X[train_mask], y[train_mask], X[test_mask], y[test_mask], X[val_mask], y[val_mask]

Wall time: 39.7 s


In [3]:
# нормируем данные
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

#### Шаг 1: учим классификаторы с семинара

Вообще говоря, мы можем научить классификаторы с базовыми гиперпараметрами -- на семинаре мы видели, что они показывают на нашей задаче неплохое качество, но ведь нет предела совершенству -- давайте подберём какие-нибудь гиперпараметры для логрега (список можно посмотреть тут: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

#### Автоматизировать подбор гиперпараметров , как правило, удобнее -- ручной подбор предпочтителен для понимания, что и как влияет на качество моделей, но обычно занимает слишком много времени и сил без каких-либо преимуществ над автоматическим отбором

Реализуем функцию grid_search, которая будет делать следующее: принимая обучающую и валидационную выборки, функция обучает набор из классификаторов со всевозможными комбинациями гиперпараметров (предварительно указанных нами для перебора в словаре) и выбирает из них тот, чьё качество по целевой метрике на валидационной выборке оказывается наилучшим.

$\textbf{Вопрос для размышления:}$ можно ли тут попробовать "встроить" защиту от переобучения?

In [ ]:
#реализуем функцию для подбора гиперпараметров модели
#для перехода от словаря параметров к списку комбинаций может быть полезен sklearn.model_selection.ParameterGrid
def grid_search(X_train,
                X_val,
                y_train,
                y_val,
                params_dict):
    '''
    Функция подбирает гиперпараметры мультиномиальной логитиеской регрессии для получения максимального значения accuracy
    на валидационной выборке, принимает:
    X_train -- DataFrame независимых переменных на обучающей выборке
    X_val -- DataFrame независимых переменных на валидационной выборке
    y_train -- Series таргета на обучающей выборке
    y_val -- Series таргета на валидационной выборке
    params_dict -- словарь гиперпараметров в формате {'paramater_nm':[value_1, value_2, ...]}
    '''
    ...
    
    return(best_model) #ну или best_parameters, если вдруг так интереснее

In [ ]:
# тут обучаем свой классификатор -- можно просто .fit() без подбора параметров, можно -- с подбором
...

#### Шаг 2: бинаризуем таргет и учим классификаторы

In [ ]:
def make_binary_predictors(y):
    '''
    Функция принимает Series y c категориальной переменной и делает DataFrame с [log_2(L+1)] столбцами из 0 и 1
    
    подсказка: в нашем конкретном случае можно переводить десятичное число в двоичное 
    '''
    ...
    return(targets)

In [ ]:
class BinarisedTargetClassifier():
    '''
    класс BinarisedTargetClassifier -- мультиклассовый классификатор на основании нескольких бинарных логистических регрессий
    '''
    def __init__():
        ...
    def fit(X_train, y_train):
        ...
    def predict(X):
        ...
    def predict_score(X): #(без него не построить AUC, но в целом обойтись можно)
        pass

In [ ]:
#учим созданный классификатор
from sklearn.metrics import accuracy_score

clf_bin = BinarisedTargetClassifier()
#обучаем
clf_bin.fit(X_train, y_train)
#предсказываем класс, считаем accuracy
y_pred = clf_bin.predict(X_test)
accuracy_bin = accuracy_score(y_test, y_pred)
#предсказываем вероятность, считаем AUC
score_bin = clf_bin.predict_score(X_test)
auc_bin = roc_auc_score(y_test, score_bin, average='macro', multi_class = 'ovr')

#### Шаг 3: сравнение качества полученных классификаторов

In [ ]:
accuracy_test = clf.score(X_test, y_test)
y_score = clf.predict_proba(X_test)
auc_test = roc_auc_score(y_test, y_score, average='macro', multi_class = 'ovr')

In [ ]:
res = pd.DataFrame(
{'regression_type' : ['sample',...],
'accuracy' : [accuracy_test,...],
'macro AUC' : [auc_test,...]
}
)
res

$\textbf{Вывод}:$